# Evrişimsel Sinir Ağları (Convolutional Neural Network (CNN))

Bu çalışma dosyasında, [CIFAR görüntülerini](https://www.cs.toronto.edu/~kriz/cifar.html) sınıflandırmak için basit bir [Evrişimsel Sinir Ağı (CNN)]((https://developers.google.com/machine-learning/glossary/#convolutional_neural_network)) uygulamasını içerir. Bu çalışmada [Keras Sequential API](https://www.tensorflow.org/guide/keras/overview) kullandığından, modelinizi oluşturmak ve eğitmek yalnızca birkaç satır kod alacaktır.

### TensorFlow Kütüphanesini Ekleme

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### CIFAR10 Veri Kümesini İndirme ve Ön İşlemler

CIFAR10 veri kümesi, her sınıfta 6.000 görüntü olmak üzere 10 sınıfta 60.000 renkli görüntü içerir. Veri kümesi 50.000 eğitim görüntüsüne ve 10.000 test görüntüsüne bölünmüştür. Sınıflar aralarında örtüşme yoktur.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Verileri Doğrulayın

Veri kümesinin doğru göründüğünü doğrulamak için, eğitim kümesindeki ilk 25 görüntüyü çizelim ve her görüntünün altında sınıf adını görüntüleyelim:


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Evrişimsel Temel Model Oluşturun

Aşağıdaki 6 kod satırı, [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) ve [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) katmanları kullanılarak basit bir evrişimsel model tanımlar: 

Bir CNN, girdi olarak batch size'ı göz ardı ederek, tensör (image_height, image_width, color_channels) alır.
Bu boyutlarda yeniyseniz, color_channels (R,G,B) anlamına gelir. Bu örnekte, CNN'nizi, CIFAR görüntülerinin formatı olan şekil girişlerini (32, 32, 3) işlemek için yapılandıracaksınız. 
Bunu input_shape argümanını ilk katmanınıza ileterek yapabilirsiniz.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Şimdiye kadar modelinizin mimarisini gösterelim:

In [ ]:
model.summary()

Yukarıda, her Conv2D ve MaxPooling2D katmanının çıktısının bir 3B şekil tensörü (yükseklik, genişlik, kanallar) olduğunu görebilirsiniz. Ağda daha derine indikçe genişlik ve yükseklik boyutları küçülme eğilimindedir. Her Conv2D katmanı için çıktı kanallarının sayısı, ilk argüman tarafından kontrol edilir (örneğin, 32 veya 64). Tipik olarak, genişlik ve yükseklik küçüldükçe, her Conv2D katmanına daha fazla çıktı kanalı eklemeyi (hesaplamalı olarak) karşılayabilirsiniz.

### Üstüne Dense katmanlar ekleyin

Modeli tamamlamak için, sınıflandırma gerçekleştirmek için evrişimli tabandan (şekil (4, 4, 64)) son çıktı tensörünü bir veya daha fazla Dense katmana besleyeceksiniz. Dense katmanlar, giriş olarak vektörleri (1B olan) alırken, mevcut çıktı bir 3B tensördür. İlk olarak, 3B çıktıyı 1B olarak düzleştirecek, ardından üstüne bir veya daha fazla Dense katman ekleyeceksiniz. CIFAR'ın 10 çıktı sınıfı vardır, bu nedenle 10 çıktılı son bir Dense katman kullanırsınız.

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

İşte modelinizin tam mimarisi:

In [ ]:
model.summary()

Ağ özeti, (4, 4, 64) çıktılarının, iki Dense katmandan geçmeden önce şekil vektörlerine (1024) düzleştirildiğini gösterir.

### Modeli Derleyin ve Eğitin

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### Modeli Değerlendirin

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)


Basit CNN'niz %70'in üzerinde bir test doğruluğu elde etti. Birkaç satır kod için fena değil! Başka bir CNN stili için, Keras alt sınıflama API'sini ve `tf.GradientTape`i kullanan [uzmanlar için TensorFlow 2 hızlı başlangıç](https://www.tensorflow.org/tutorials/quickstart/advanced) örneğine bakın.